
 Generates address.csv file with addresses (Address_id for customers start with address_id=36 because address_id's 1-5                reserved for cafe_id;address_id's 6-35 reserved for employee_id)

 
 Generates customer.csv file with customers 
   where each row has 
   -address_id that are already in address.csv (address_id's 1-5 contain cafe_id;address_id's 6-35 contain employee_id)


   
 Generates payment.csv file with payments where each row has 
   -customer_id that is already in customer.csv  
 Generates orders.csv file with orders where each row has 
   -customer_id that is already in customer.csv 
   -payment_id that is already in payment.csv
   -if delivery=true than orders.delivery_address_id has address_id that are already in address.csv
     

In [60]:
from faker import Faker
import pandas as pd

fake = Faker(locale='en_US')
fake_latam= Faker(locale='es_AR') #instance of Faker for Latin American addresses


Addresses_list = []  
Customers_list=[]


#createAddresses creates a list of customer addresses
def createAddresses(num_of_addresses):
   
    # Creating an empty list to add our address dictionaries   
    # Loop through the number of addresses starting with 36 (address_id=36 is the address of the first customer)
    for i in range(36,36+num_of_addresses):  
        Address = {}  # Initialize an empty dictionary
        Address_ = {}
        Address_["address_id"] =i
        Address_["city"]=fake.random_element(elements=(
            "New York", "New York", "New York", 
            "Miami", "Miami", "Seattle", 
            "Lima", "Lima", "Buenos Aires")) 
        Address_["postal_code"] =fake.postcode()  # Use Faker's `postcode()` method
        if  Address_["city"]=="New York":
            Address_["country"]="USA"
        elif  Address_["city"]=="Miami":
              Address_["country"]="USA"
        elif  Address_["city"]=="Seattle":
              Address_["country"]="USA"
        elif  Address_["city"]=="Buenos Aires":
              Address_["country"]="Argentina"  
        elif  Address_["city"]=="Lima":
              Address_["country"]="Peru"  
            
        if Address_["country"]=="USA":          
           Address_["street_address"] =  Faker(locale='en_US').street_address()
        elif Address_["country"]=="Argentina":          
           Address_["street_address"] = fake_latam.street_address()
        elif Address_["country"]=="Peru":          
           Address_["street_address"] = fake_latam.street_address()  
            
        Address_["street_address_second_line"] = fake.building_number()
        Address_["last_update"] = str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%H:%M:%S"))

        
        #Placing all columns in correct order for our database
        Address["address_id"]=Address_["address_id"]
        Address["street_address"] =Address_["street_address"] 
        Address["street_address_second_line"] =Address_["street_address_second_line"]
        Address["city"]=Address_["city"]
        Address["postal_code"]= Address_["postal_code"]
        Address["country"]=Address_["country"]
        Address["last_update"] =Address_["last_update"]
        
        Addresses_list.append(Address)  # Add the address dictionary to the list
    return pd.DataFrame(Addresses_list)

def createCustomers(num_of_customers):
      # Initialize an empty dictionary
    for i in range(1, num_of_customers):
        Customer = {}
        Customer ["customer_id"]=i
        address_id = 35 + i
        Customer ["address_id"]=address_id  

        country = None
        #if address_id in Addresses_list:
            # Use `address_dict` if `address_id` exists in `Addresses_list`
        country =Addresses_list[i-1]["country"]  # Prevent index out of range
       # print(country)     
        if country=="USA":        
           Customer ["first_name"] = fake.first_name()       
           Customer ["last_name"]  = fake.last_name_nonbinary()
            
        if country=="Argentina" or country=="Peru" :        
           Customer ["first_name"] = fake_latam.first_name()       
           Customer ["last_name"]  = fake_latam.last_name_nonbinary()  
                  
        Customer ["phone_number"]=fake.msisdn()
        Customer ["email"]=fake.ascii_free_email()
        Customer ["active"]=fake.random_element(elements=("true","true","true","true","false"))
        Customer ["last_update"]= str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X"))
        
        Customers_list.append(Customer)  # Add the address dictionary to the list

    return pd.DataFrame(Customers_list)


data_addresses=createAddresses(999)
data_addresses.to_csv('Addresses_of_customers.csv',index=False)

data_customers=createCustomers(999)
data_customers.to_csv('Customers.csv',index=False)

#print(Addresses_list[8]) 